<a href="https://colab.research.google.com/github/chandan110791/NLP/blob/main/2_Syntactic_Processing_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## We will pre process the reviews and get information from them using POS tags

#Importing Libraries

import pandas as pd
import numpy as np
import os 

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
#Read Review Data

con = open("Samsung.txt","r",encoding="utf-8")
samsung_review=con.read()
con.close()

In [67]:
samsung_review.split("\n")[0:10]

["I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!",
 'nice phone, nice up grade from my pantach revue. Very clean set up and easy set up. never had an android phone but they are fantastic to say the least. perfect size for surfing and social media. great phone samsung',
 'Very pleased',
 'It works good but it goes slow sometimes but its a very good phone I love it',
 'Great phone to replace my lost phone. The only thing is the volume up button does not work, but I can still go into settings to adjust. Other than that, it does the job until I am eligible to upgrade my phone again.Thaanks!',
 'I originally was using the Samsung S2 Galaxy for Sprint and wanted to retu

In [8]:
#Dataset is a text file where each review is in a new line

In [9]:
#Will our hypothesis hold on real world data? Product features---POS_NOUN

In [11]:
review_1 = samsung_review.split("\n")[0]

tokens = nlp(review_1)

for token in tokens:
  print(token.text+"|"+token.pos_)

I|PRON
feel|VERB
so|SCONJ
LUCKY|NOUN
to|PART
have|AUX
found|VERB
this|PRON
used|VERB
(|PUNCT
phone|NOUN
to|ADP
us|PRON
&|CCONJ
not|PART
used|VERB
hard|ADV
at|ADV
all|ADV
)|PUNCT
,|PUNCT
phone|NOUN
on|ADP
line|NOUN
from|ADP
someone|PRON
who|PRON
upgraded|VERB
and|CCONJ
sold|VERB
this|DET
one|NUM
.|PUNCT
My|PRON
Son|PROPN
liked|VERB
his|PRON
old|ADJ
one|NOUN
that|PRON
finally|ADV
fell|VERB
apart|ADV
after|ADP
2.5|NUM
+|NUM
years|NOUN
and|CCONJ
did|AUX
n't|PART
want|VERB
an|DET
upgrade|NOUN
!|PUNCT
!|PUNCT
Thank|VERB
you|PRON
Seller|PROPN
,|PUNCT
we|PRON
really|ADV
appreciate|VERB
it|PRON
&|CCONJ
your|PRON
honesty|NOUN
re|PROPN
:|PUNCT
said|VERB
used|ADJ
phone|NOUN
.|PUNCT
I|PRON
recommend|VERB
this|DET
seller|NOUN
very|ADV
highly|ADV
&|CCONJ
would|AUX
but|CCONJ
from|ADP
them|PRON
again|ADV
!|PUNCT
!|PUNCT


In [12]:
#Real world data is usually messy, observe the words found and used

In [17]:
pos = []
lemma = []
text = []
for tok in tokens:
    pos.append(tok.pos_)
    lemma.append(tok.lemma_)
    text.append(tok.text)

nlp_table = pd.DataFrame({"text":text,"lemma":lemma,"pos":pos})
nlp_table.head(10)

,text,lemma,pos
0,I,I,PRON
1,feel,feel,VERB
2,so,so,SCONJ
3,LUCKY,lucky,NOUN
4,to,to,PART
5,have,have,AUX
6,found,find,VERB
7,this,this,PRON
8,used,use,VERB
9,(,(,PUNCT


In [20]:
nlp_table[nlp_table.pos=="NOUN"]["lemma"].value_counts()

phone      3
lucky      1
line       1
one        1
year       1
upgrade    1
honesty    1
seller     1
Name: lemma, dtype: int64

In [21]:
#It seems possible that if we extract all the nouns from the reviews and look at the top 5 most frequent lemmatised noun forms, we will be able to identify What people are talking about?

In [22]:
from tqdm import tqdm
nouns = []

for reviews in tqdm(samsung_review.split("\n")[0:1000]):
  for token in  nlp(reviews):
    if token.pos_=="NOUN":
        nouns.append(token.lemma_)

pd.Series(nouns).value_counts().head(5)

100%|██████████| 1000/1000 [00:12<00:00, 82.04it/s]


phone      1217
time         92
battery      92
screen       86
price        86
dtype: int64

In [28]:
#Did you notice anything? What do you think will be the time taken to process all the reviews?

(len(samsung_review.split("\n"))/1000*17)/60

13.133916666666666

**Summary**

> POS tag based rule seems to be working well

> We need to figure out a way to reduce the time taken to process reviews



In [29]:
# shorten the pipline loading
nlp=spacy.load('en_core_web_sm',disable=['parser','ner'])

In [30]:
from tqdm import tqdm
nouns = []

for reviews in tqdm(samsung_review.split("\n")[0:1000]):
  for token in  nlp(reviews):
    if token.pos_=="NOUN":
        nouns.append(token.lemma_)

100%|██████████| 1000/1000 [00:05<00:00, 179.45it/s]


In [33]:
(len(samsung_review.split("\n"))/1000*5)/60

3.862916666666666

In [34]:
##Lets process all the reviews now and see if time taken is less !!!
from tqdm import tqdm
nouns = []

for reviews in tqdm(samsung_review.split("\n")):
  for token in  nlp(reviews):
    if token.pos_=="NOUN":
        nouns.append(token.lemma_)

100%|██████████| 46355/46355 [03:39<00:00, 211.28it/s]


In [35]:
pd.Series(nouns).value_counts().head(10)

phone      43650
battery     4362
product     3912
time        3828
screen      3816
card        3373
price       3148
problem     3134
camera      2911
app         2577
dtype: int64

We now know that people mention battery, product, screen etc. But we still don't know in what context they mention these keywords


> Summary:


Most frequently used lemmatised forms of noun, inform us about the product features people are talking about in product reviews
In order to process the review data faster spacy allows us to use the idea of enabling parts of model inference pipeline via spacy.loads() command and disable parameter






> We can use a simple hueristic


Find out what were the most common words that appeared before and after each mention of product feature
Use regex pattern to extract this information

In [36]:
import re

In [37]:
pattern = re.compile("\w+\sbattery\s\w+")

In [42]:
s1 = "the battery was great"
result = re.findall(pattern,s1)

In [43]:
result

['the battery was']

In [46]:
result[0].split(" ")[0]

'the'

In [47]:
result[0].split(" ")[-1]

'was'

In [48]:
#Extract all the prefixes and suffixes of battery
result = re.findall(pattern,samsung_review)

In [49]:
prefix = []
suffix = []
for token in result:
  temp = token.split(" ")
  prefix.append(temp[0])
  suffix.append(temp[-1])

In [54]:
pd.Series(prefix).value_counts().head(10)

the          1060
The           336
good           92
and            82
great          61
new            50
removable      48
better         38
of             38
a              38
dtype: int64

In [55]:
pd.Series(suffix).value_counts().head(10)

life     1052
is        210
and       146
lasts      83
was        66
last       53
doesn      40
does       38
in         34
that       33
dtype: int64

This doesn't make much sense as these are commonly used words. Let's remove stopwords and see what we get

In [56]:
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]


In [58]:
prefix = [p for p in prefix if p not in stop_words]
suffix = [s for s in suffix if s not in stop_words]

In [59]:
prefix=pd.Series(prefix).value_counts().head(5).index
suffix=pd.Series(suffix).value_counts().head(5).index

In [61]:
pd.DataFrame({'prefixes':prefix,'keyword':['battery']*len(prefix),'suffixes':suffix})

,prefixes,keyword,suffixes
0,The,battery,life
1,good,battery,lasts
2,great,battery,last
3,new,battery,doesn
4,removable,battery,runs


In [62]:
# Lets put all this logic in a function

def get_context(reviews,keyword):
    pattern = re.compile(f"\w+\s{keyword}\s\w+")
    prefixes_suffixes = re.findall(pattern,reviews)
    prefixes = []
    suffixes = []
    for p in prefixes_suffixes:
        l = p.split(" ")
        prefixes.append(l[0].lower())
        suffixes.append(l[-1].lower())
    prefixes = [p for p in prefixes if p not in stop_words]
    suffixes = [s for s in suffixes if s not in stop_words]
    prefixes=pd.Series(prefixes).value_counts().head(5).index
    suffixes=pd.Series(suffixes).value_counts().head(5).index
    return pd.DataFrame({'prefixes':prefixes,'keyword':[f'{keyword}']*len(prefixes),'suffixes':suffixes})

In [64]:
get_context(samsung_review,"battery")


,prefixes,keyword,suffixes
0,good,battery,life
1,great,battery,lasts
2,long,battery,last
3,new,battery,doesn
4,removable,battery,runs


In [66]:
get_context(samsung_review,"screen")

,prefixes,keyword,suffixes
0,touch,screen,protector
1,big,screen,size
2,great,screen,resolution
3,large,screen,protectors
4,home,screen,quality


> Summary:

- Simple hueristics sometime are very usefull
- Regex can be life saviours
- Don't forget to use simple text processing while trying to solve a non-trival problem